In [82]:
import torch
import torch.nn as nn
import torchvision
from torch.autograd import Variable
import torchvision.transforms as transforms

import pandas as pd
from sklearn.model_selection import train_test_split

test_size = 0.33
batch_size = 128

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

d_frame = pd.read_csv(path, sep=",", index_col=None)

dataset = d_frame.values

test_dataset = dataset[0:int(test_size * len(dataset)),:]
train_dataset = dataset[int(test_size * len(dataset)):,:]

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [131]:
n_output_neuron = 1
n_input_neuron = X_train.shape[1]
hidden_size = 100
learning_rate = 0.0001

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    
model = NeuralNet(n_input_neuron, hidden_size, n_output_neuron).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [134]:
num_epochs = 50
total_step = len(train_loader)


for epoch in range(num_epochs):
    loss_acc = np.zeros(len(train_loader))
    for i, features in enumerate(train_loader):  

        target = features[:,0]
        feature = features[:,1:]
        
        # Move tensors to the configured device
        target = target.to(device).float()
        feature = feature.to(device).float()
        
        # Forward pass
        output = model(feature)
        
        # Compute loss
        loss = criterion(output, target)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_acc[i] = loss.item()
    print ('Epoch [{}], Loss: {:.4f}' .format(epoch+1, loss_acc.mean()))
    print()
            


Epoch [1], Loss: 1117.0199

Epoch [2], Loss: 1014.4366

Epoch [3], Loss: 924.3578

Epoch [4], Loss: 845.8578

Epoch [5], Loss: 758.5863

Epoch [6], Loss: 672.7929

Epoch [7], Loss: 617.3023

Epoch [8], Loss: 548.1136

Epoch [9], Loss: 485.9874

Epoch [10], Loss: 456.2655

Epoch [11], Loss: 406.9650

Epoch [12], Loss: 364.7205

Epoch [13], Loss: 352.0361

Epoch [14], Loss: 309.8224

Epoch [15], Loss: 282.7503

Epoch [16], Loss: 264.3343

Epoch [17], Loss: 253.4604

Epoch [18], Loss: 208.1712

Epoch [19], Loss: 218.0318

Epoch [20], Loss: 248.4261

Epoch [21], Loss: 237.0577

Epoch [22], Loss: 747.8049

Epoch [23], Loss: 2325.5877

Epoch [24], Loss: 2681.2012

Epoch [25], Loss: 608.6621

Epoch [26], Loss: 3006.9947

Epoch [27], Loss: 2341.8255

Epoch [28], Loss: 570.5800

Epoch [29], Loss: 2810.9108

Epoch [30], Loss: 746.1948

Epoch [31], Loss: 5527.6618

Epoch [32], Loss: 320.6918

Epoch [33], Loss: 3411.4033

Epoch [34], Loss: 661.9382

Epoch [35], Loss: 7100.8730

Epoch [36], Loss: 1

In [125]:
# test the trained model
import numpy as np
ms_error = list()

with torch.no_grad():
    for test_item in test_loader:
        feature = test_item[:,1:]
        target = test_item[:,0]
        
        target = target.to(device).float()
        feature = feature.to(device).float()
        
        output = model(feature)
        output = torch.reshape(output, (output.size(0),))

        ms_error.append(((output-target)**2).mean())

print("Mean squared error: %.4f" % np.mean(ms_error))

Mean squared error: 51493.5391
